In [ ]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten, LSTM
import time

# 데이터셋 로드
data = pd.read_csv("voice.csv")

# 데이터 인코딩 (문자열로 된 레이블을 숫자로 변환: 모델이 이해할 수 있도록)
label_map = {'male': 0, 'female': 1}
data['label'] = data['label'].replace(label_map)

# X, y 분리
X = data.drop('label', axis=1).copy()
y = data['label'].copy()

# 데이터 정규화 (data preprocessing: zero-center + normalize)
X = (X - X.mean()) / X.std()

In [ ]:
# 데이터를 train data, test data 로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)#, random_state=42) # 호출할 때마다 동일한 train, test dataset를 생성하기 위한 난수 값

## CNN

In [ ]:
# CNN 모델 정의
cnn_model = tf.keras.Sequential([
    tf.keras.layers.Reshape((20, 1), input_shape=(20,)), # 입력 데이터의 형태 변경. 1차원 -> 2차원 (Conv1D 레이어 입력 형태)
    tf.keras.layers.Conv1D(512, 5, activation=tf.keras.layers.LeakyReLU(alpha=0.2)), # filters=32, kernel_size=3 / 일반적으로 Conv1D 레이어에서 필터의 개수를 2의 거듭제곱수로 지정하는 것이 좋음
    tf.keras.layers.MaxPooling1D(2), # pool_size=2
    # tf.keras.layers.Conv1D(256, kernel_size=3, activation=tf.keras.layers.LeakyReLU(alpha=0.2)),
    # tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation=tf.keras.layers.LeakyReLU(alpha=0.2)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# learning rate 조절
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001) # 기본 learning rate: 0.001

# 모델 컴파일
cnn_model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
# 모델의 구조 요약 및 출력
cnn_model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_10 (Reshape)        (None, 20, 1)             0         
                                                                 
 conv1d_5 (Conv1D)           (None, 16, 512)           3072      
                                                                 
 max_pooling1d_5 (MaxPoolin  (None, 8, 512)            0         
 g1D)                                                            
                                                                 
 flatten_5 (Flatten)         (None, 4096)              0         
                                                                 
 dense_50 (Dense)            (None, 64)                262208    
                                                                 
 dropout_15 (Dropout)        (None, 64)                0         
                                                     

In [ ]:
# 콜백 함수 정의 및 모델 학습
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

start_time = time.time()

cnn_model.fit(X_train, y_train, epochs=200, validation_data=(X_test, y_test), batch_size=64, shuffle=True, callbacks=[early_stopping])

end_time = time.time()
training_time = end_time - start_time
print("CNN 모델의 총 학습 시간:", training_time, "초")

Epoch 1/200
40/40 [==============================] - 2s 20ms/step - loss: 0.2507 - accuracy: 0.8927 - val_loss: 0.1174 - val_accuracy: 0.9669
Epoch 2/200
40/40 [==============================] - 1s 14ms/step - loss: 0.0999 - accuracy: 0.9641 - val_loss: 0.0655 - val_accuracy: 0.9842
Epoch 3/200
40/40 [==============================] - 1s 14ms/step - loss: 0.0787 - accuracy: 0.9743 - val_loss: 0.0596 - val_accuracy: 0.9842
Epoch 4/200
40/40 [==============================] - 1s 15ms/step - loss: 0.0725 - accuracy: 0.9751 - val_loss: 0.0582 - val_accuracy: 0.9795
Epoch 5/200
40/40 [==============================] - 1s 13ms/step - loss: 0.0683 - accuracy: 0.9767 - val_loss: 0.0631 - val_accuracy: 0.9763
Epoch 6/200
40/40 [==============================] - 1s 15ms/step - loss: 0.0620 - accuracy: 0.9767 - val_loss: 0.0724 - val_accuracy: 0.9763
Epoch 7/200
40/40 [==============================] - 1s 14ms/step - loss: 0.0590 - accuracy: 0.9783 - val_loss: 0.0648 - val_accuracy: 0.9779
Epoch 

In [ ]:
# 모델 평가
print("CNN model: ")
cnn_model.evaluate(X_test,  y_test, verbose=2)

CNN model: 
20/20 - 0s - loss: 0.0491 - accuracy: 0.9748 - 67ms/epoch - 3ms/step


[0.04910450428724289, 0.9747633934020996]

## RNN

In [ ]:
# RNN 모델 정의
rnn_model = tf.keras.Sequential([
    tf.keras.layers.Reshape((20, 1), input_shape=(20,)),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# 모델 컴파일
rnn_model.compile(optimizer='adam', # 기본 learning rate: 0.001
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# 모델의 구조 요약 및 출력
rnn_model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_11 (Reshape)        (None, 20, 1)             0         
                                                                 
 lstm_5 (LSTM)               (None, 64)                16896     
                                                                 
 dense_52 (Dense)            (None, 64)                4160      
                                                                 
 dense_53 (Dense)            (None, 1)                 65        
                                                                 
Total params: 21121 (82.50 KB)
Trainable params: 21121 (82.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# 콜백 함수 정의 및 모델 학습
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

start_time = time.time()

rnn_model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), shuffle=True, callbacks=[early_stopping])

end_time = time.time()
training_time = end_time - start_time
print("RNN 모델의 총 학습 시간:", training_time, "초")

Epoch 1/100
80/80 [==============================] - 5s 24ms/step - loss: 0.5775 - accuracy: 0.6882 - val_loss: 0.4191 - val_accuracy: 0.8233
Epoch 2/100
80/80 [==============================] - 1s 11ms/step - loss: 0.4061 - accuracy: 0.8149 - val_loss: 0.3299 - val_accuracy: 0.8533
Epoch 3/100
80/80 [==============================] - 1s 11ms/step - loss: 0.3523 - accuracy: 0.8358 - val_loss: 0.2868 - val_accuracy: 0.8612
Epoch 4/100
80/80 [==============================] - 1s 11ms/step - loss: 0.3153 - accuracy: 0.8548 - val_loss: 0.2729 - val_accuracy: 0.8991
Epoch 5/100
80/80 [==============================] - 1s 11ms/step - loss: 0.2949 - accuracy: 0.8721 - val_loss: 0.2514 - val_accuracy: 0.9006
Epoch 6/100
80/80 [==============================] - 1s 11ms/step - loss: 0.2666 - accuracy: 0.8895 - val_loss: 0.2439 - val_accuracy: 0.8991
Epoch 7/100
80/80 [==============================] - 1s 11ms/step - loss: 0.2602 - accuracy: 0.8911 - val_loss: 0.2176 - val_accuracy: 0.9180
Epoch 

In [ ]:
# 모델 평가
print("RNN model: ")
rnn_model.evaluate(X_test,  y_test, verbose=2)

RNN model: 
20/20 - 0s - loss: 0.1105 - accuracy: 0.9543 - 93ms/epoch - 5ms/step


[0.11045819520950317, 0.9542586803436279]

## MLP

In [ ]:
# MLP 모델 정의
mlp_model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# learning rate 조절
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001) # 기본 learning rate: 0.001

# 모델 컴파일
mlp_model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# 모델의 구조 요약 및 출력
mlp_model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_54 (Dense)            (None, 256)               5376      
                                                                 
 dense_55 (Dense)            (None, 128)               32896     
                                                                 
 dropout_16 (Dropout)        (None, 128)               0         
                                                                 
 dense_56 (Dense)            (None, 32)                4128      
                                                                 
 dense_57 (Dense)            (None, 64)                2112      
                                                                 
 dropout_17 (Dropout)        (None, 64)                0         
                                                                 
 dense_58 (Dense)            (None, 32)              

In [ ]:
# 콜백 함수 정의 및 모델 학습
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

start_time = time.time()

mlp_model.fit(X_train, y_train, epochs=200, validation_data=(X_test, y_test), batch_size=64, shuffle=True, callbacks=[early_stopping])

end_time = time.time()
training_time = end_time - start_time
print("MLP 모델의 총 학습 시간:", training_time, "초")

Epoch 1/200
40/40 [==============================] - 2s 9ms/step - loss: 0.3146 - accuracy: 0.8911 - val_loss: 0.0629 - val_accuracy: 0.9795
Epoch 2/200
40/40 [==============================] - 0s 4ms/step - loss: 0.0990 - accuracy: 0.9704 - val_loss: 0.0697 - val_accuracy: 0.9748
Epoch 3/200
40/40 [==============================] - 0s 4ms/step - loss: 0.0720 - accuracy: 0.9751 - val_loss: 0.0632 - val_accuracy: 0.9795
Epoch 4/200
40/40 [==============================] - 0s 4ms/step - loss: 0.0674 - accuracy: 0.9803 - val_loss: 0.0662 - val_accuracy: 0.9779
Epoch 5/200
40/40 [==============================] - 0s 4ms/step - loss: 0.0608 - accuracy: 0.9799 - val_loss: 0.0593 - val_accuracy: 0.9826
Epoch 6/200
40/40 [==============================] - 0s 4ms/step - loss: 0.0557 - accuracy: 0.9815 - val_loss: 0.0692 - val_accuracy: 0.9748
Epoch 7/200
40/40 [==============================] - 0s 5ms/step - loss: 0.0489 - accuracy: 0.9854 - val_loss: 0.0776 - val_accuracy: 0.9700
Epoch 8/200
4

In [ ]:
# 모델 평가
print("MLP model: ")
mlp_model.evaluate(X_test,  y_test, verbose=2)

MLP model: 
20/20 - 0s - loss: 0.0567 - accuracy: 0.9811 - 44ms/epoch - 2ms/step


[0.05671234056353569, 0.9810725450515747]